In [53]:
from tensorflow import keras 
import numpy as np 

In [56]:
from keras.layers import Dense
model = keras.models.Sequential()
model.add(Dense(12, input_dim=2, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [ ]:
model.predict(X_test, batch_size=32)
model.predict_classes(X_test, batch_size=32)